In [172]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import pandas as pd
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import KFold
from torch.utils.data import Subset
from tqdm import tqdm

os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

In [173]:
with open("nepali_characters.txt", encoding="utf-8") as f:
    for i, line in enumerate(f):
        print(f"Line {i}: {repr(line)}")

Line 0: '\n'
Line 1: ' \n'
Line 2: '!\n'
Line 3: '"\n'
Line 4: '(\n'
Line 5: ')\n'
Line 6: '-\n'
Line 7: '.\n'
Line 8: '/\n'
Line 9: '0\n'
Line 10: '1\n'
Line 11: '2\n'
Line 12: '3\n'
Line 13: '4\n'
Line 14: '5\n'
Line 15: '6\n'
Line 16: '7\n'
Line 17: '8\n'
Line 18: '9\n'
Line 19: ':\n'
Line 20: '=\n'
Line 21: '?\n'
Line 22: 'A\n'
Line 23: 'B\n'
Line 24: 'C\n'
Line 25: 'D\n'
Line 26: 'E\n'
Line 27: 'F\n'
Line 28: 'G\n'
Line 29: 'H\n'
Line 30: 'I\n'
Line 31: 'K\n'
Line 32: 'L\n'
Line 33: 'M\n'
Line 34: 'N\n'
Line 35: 'O\n'
Line 36: 'P\n'
Line 37: 'Q\n'
Line 38: 'R\n'
Line 39: 'S\n'
Line 40: 'T\n'
Line 41: 'U\n'
Line 42: 'W\n'
Line 43: 'Y\n'
Line 44: 'a\n'
Line 45: 'b\n'
Line 46: 'c\n'
Line 47: 'd\n'
Line 48: 'e\n'
Line 49: 'f\n'
Line 50: 'g\n'
Line 51: 'h\n'
Line 52: 'i\n'
Line 53: 'j\n'
Line 54: 'k\n'
Line 55: 'l\n'
Line 56: 'm\n'
Line 57: 'n\n'
Line 58: 'o\n'
Line 59: 'p\n'
Line 60: 'r\n'
Line 61: 's\n'
Line 62: 't\n'
Line 63: 'u\n'
Line 64: 'v\n'
Line 65: 'w\n'
Line 66: 'y\n'
Line 6

In [174]:

def load_labels_safely(csv_path, encoding='utf-8'):
    rows = []
    with open(csv_path, 'r', encoding=encoding) as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split(' ', 1)  # split at the first space only
            if len(parts) == 2:
                filename, text = parts
                rows.append((filename, text))
    return pd.DataFrame(rows, columns=["filename", "text"])

In [175]:
# Dataset Loading
class NepaliOCRDataset(Dataset):
    def __init__(self, csv_path, image_dir, transform=None, charset=None):
        self.data = load_labels_safely(csv_path, encoding='utf-16')
        self.image_dir = image_dir
        self.transform = transform or transforms.Compose([
            transforms.Grayscale(),
            transforms.Resize((16,128)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, ), (0.5,))
        ])
        self.charset = charset
        self.char_to_idx = {char: idx + 1 for idx, char in enumerate(self.charset)}
    
    def __len__(self):
        return len(self.data)
    
    def encode_text(self, text):
        return torch.tensor([self.char_to_idx[c] for c in text], dtype=torch.long)
    
    def __getitem__(self, index):
        img_name = self.data.iloc[index, 0]
        label = self.data.iloc[index, 1]
        img_path = os.path.join(self.image_dir, img_name)
        image = Image.open(img_path).convert("RGB")
        image = self.transform(image)
        label_encoded = self.encode_text(label)
        return image, label_encoded


In [176]:
# Defining CRNN Model

class CRNN(nn.Module):
    def __init__(self, num_classes):
        super(CRNN, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2,2),
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2,2),
            nn.Conv2d(128, 256, 3, padding=1), nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1), nn.ReLU(), nn.MaxPool2d((2,1), (2,1)),
            nn.Conv2d(256, 512, 3, padding=1), nn.ReLU(),
            nn.BatchNorm2d(512),
            nn.Conv2d(512, 512, 3, padding=1), nn.ReLU(), nn.BatchNorm2d(512), nn.MaxPool2d((2,1), (2,1))
        )
        self.rnn = nn.LSTM(512, 256, bidirectional=True, num_layers=2, batch_first=True)
        self.fc = nn.Linear(512, num_classes + 1) # +1 for CTC blank

    def forward(self,x):
        x = self.cnn(x)
        b, c, h, w = x.size()
        assert h == 1, "height must be 1 after conv layers"
        x = x.squeeze(2).permute(0,2,1)     # [batch, width, channels]
        x, _ = self.rnn(x)
        x = self.fc(x)
        return x.permute(1,0,2)     # [T, B, C] for CTC

In [177]:
# Collate Function

def collate_fn(batch):
    images, labels = zip(*batch)
    image_batch = torch.stack(images, 0)
    label_lengths = torch.tensor([len(l) for l in labels], dtype=torch.long)
    labels = pad_sequence(labels, batch_first=True, padding_value=0)
    return image_batch, labels, label_lengths

In [178]:
# Training Loop

def train(k=5):
    # Device selection with MPS support
    if torch.backends.mps.is_available() and torch.backends.mps.is_built():
        device = torch.device("mps")
    elif torch.cuda.is_available():
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")
    print(f"Using Device: {device}")

    charset = [line.rstrip('\n') for line in open("nepali_characters.txt", encoding="utf-8")]

    full_dataset = NepaliOCRDataset("./dataset/labels.csv", "./dataset/train", charset=charset)
    kf = KFold(n_splits=k, shuffle=True, random_state=42)

    for fold, (train_idx, val_idx) in enumerate(kf.split(full_dataset)):
        print(f"\n--- Fold {fold + 1} ---")

        train_subset = Subset(full_dataset, train_idx)
        val_subset = Subset(full_dataset, val_idx)

        train_loader = DataLoader(train_subset, batch_size=8, shuffle=True, collate_fn=collate_fn)
        val_loader = DataLoader(val_subset, batch_size=8, shuffle=False, collate_fn=collate_fn)

        model = CRNN(num_classes=len(charset)).to(device)
        criterion = nn.CTCLoss(blank= 0, zero_infinity=True)
        optimizer = optim.Adam(model.parameters(), lr=0.001)

        for epoch in range(50):
            model.train()
            train_loss = 0.0

            # tqdm wrapper here
            for images, labels, label_lengths in tqdm(train_loader, desc=f"Epoch {epoch+1} [Train]"):
                images = images.to(device)
                labels = labels.to(device)

                preds = model(images)
                input_lengths = torch.full((preds.size(1),), preds.size(0), dtype=torch.long, device=device)

                loss = criterion(preds.log_softmax(2).cpu(), labels.cpu(), input_lengths.cpu(), label_lengths.cpu())

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                train_loss += loss.item()

            # validation loop with tqdm
            model.eval()
            val_loss = 0.0
            with torch.no_grad():
                for images, labels, label_lengths in tqdm(val_loader, desc=f"Epoch {epoch+1} [Val]"):
                    images = images.to(device)
                    labels = labels.to(device)

                    preds = model(images)
                    input_lengths = torch.full((preds.size(1),), preds.size(0), dtype=torch.long, device=device)
                    loss = criterion(
                        preds.log_softmax(2).cpu(),
                        labels.cpu(),
                        input_lengths.cpu(),
                        label_lengths.cpu()
                    )

                    val_loss += loss.item()

            print(f"Epoch {epoch+1}: Train Loss = {train_loss/len(train_loader):.4f}, Val Loss = {val_loss/len(val_loader):.4f}")

        torch.save(model.state_dict(), f"crnn_fold{fold+1}.pth")
        print(f"Saved Fold {fold+1} model.\n")

if __name__ == "__main__":
    train(k=5)


Using Device: mps

--- Fold 1 ---


Epoch 1 [Val]: 100%|██████████| 109/109 [00:01<00:00, 92.75it/s]


Epoch 1: Train Loss = 4.2972, Val Loss = 3.9362


Epoch 2 [Val]: 100%|██████████| 109/109 [00:01<00:00, 92.45it/s]


Epoch 2: Train Loss = 3.8727, Val Loss = 3.8423


...

Epoch 45: Train Loss = 0.2997, Val Loss = 3.7922


Epoch 46 [Val]: 100%|██████████| 109/109 [00:01<00:00, 95.43it/s]


Epoch 46: Train Loss = 0.2712, Val Loss = 3.8795


Epoch 47 [Train]:   7%|▋         | 30/436 [00:00<00:13, 31.21it/s]


KeyboardInterrupt: 